In [10]:
import pyspark
from pyspark import SparkConf ,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
import xml.etree.ElementTree as ET
import pyspark.sql.functions as F


conf = pyspark.SparkConf().setAppName('App1').setMaster('local').set("spark.executor.cores", "4").set("spark.cores.max", "4").set('spark.executor.memory', '6g')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sc.setLogLevel('ERROR')
spark = SparkSession(sc)
sc1 = spark.sparkContext

In [13]:
#Read JSON file
path = "/data/informatica/shared/babynames/births-with-visitor-data.json"
##babydf1 = spark.read.option("multiline", "true").json(path)
##"county",  
babydf1 = spark.read.json(path)

In [45]:
babydf1.show(2)

+---------------+-------+----------+----------+--------------------+----+----------+--------+---+------------------+----------+--------------------+----+
|_corrupt_record| county|created_at|first_name|                  id|meta|name_count|position|sex|               sid|updated_at|            visitors|year|
+---------------+-------+----------+----------+--------------------+----+----------+--------+---+------------------+----------+--------------------+----+
|           null|  KINGS|1574264158|      ZOEY|00000000-0000-000...| { }|        11|       0|  F|row-r9pv-p86t.ifsp|1574264158|<?xml version="1....|2007|
|           null|SUFFOLK|1574264158|      ZOEY|00000000-0000-000...| { }|         6|       0|  F|row-7v2v~88z5-44se|1574264158|<?xml version="1....|2007|
+---------------+-------+----------+----------+--------------------+----+----------+--------+---+------------------+----------+--------------------+----+
only showing top 2 rows



In [14]:
@F.udf('array<struct<id:string, age:string, sex:string>>')
def parse_xml(s):
    root = ET.fromstring(s)
    return list(map(lambda x: x.attrib, root.findall('visitor')))
    
df2 = babydf1.select(
    F.explode(parse_xml('visitors')).alias('visitors')
).select('visitors.*',)

df2.show()

22/04/21 19:02:42 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 3)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_2455/306775425.py", line 3, in parse_xml
  File "/data/informatica/conda/lib/python3.9/xml/etree/ElementTree.py", line 1347, in XML
    parser.feed(text)
TypeError: a bytes-like object is required, not 'NoneType'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:508)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_2455/306775425.py", line 3, in parse_xml
  File "/data/informatica/conda/lib/python3.9/xml/etree/ElementTree.py", line 1347, in XML
    parser.feed(text)
TypeError: a bytes-like object is required, not 'NoneType'
